In [1]:
import pandas as pd
import numpy as np
import sklearn.ensemble
import matplotlib.pyplot as plt
import pickle
import datetime

In [2]:
def doPCA(raw_file):
    from sklearn.decomposition import PCA
    pca = PCA(tol=0.8)
    pca.fit(raw_file)
    return pca

In [3]:
tstart = datetime.datetime.now()

In [4]:
#reading the data file
raw_file = pd.read_csv('no_NAs_A.raw', sep=" ")
raw_file.head(3)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,rs12565286_C,rs12082473_A,rs3094315_G,rs2286139_T,...,rs5770820_A,rs9628187_T,rs6010063_A,rs715586_T,rs8137951_A,rs3810648_G,rs3865766_C,rs2238837_C,rs34726907_T,rs28729663_A
0,NA06984,NA06984,0,0,0,-9,0,0,0,2,...,0,0,2,0,0,0,2,0,0,0
1,NA06985,NA06985,0,0,0,-9,0,0,0,2,...,1,0,0,0,2,1,0,1,1,1
2,NA06986,NA06986,0,0,0,-9,0,0,0,2,...,0,1,1,0,1,0,2,0,0,0


In [5]:
#deleting the columns not part of the model
raw_file = raw_file.drop(raw_file.columns[[0,1,2,3,4,5]],axis=1)

In [6]:
tfinish = datetime.datetime.now()
texecute = tfinish - tstart
print(texecute)

0:38:42.813000


In [7]:
snps_pyth = list(raw_file.columns.values)

In [8]:
len(snps_pyth)

857124

In [9]:
with open('snps_pyth','wb') as f:
    pickle.dump(snps_pyth,f)

In [10]:
tstart = datetime.datetime.now()

In [11]:
pca = doPCA(raw_file)

In [12]:
tfinish = datetime.datetime.now()
texecute = tfinish - tstart
print(texecute)

0:01:21.328000


In [13]:
snps_pyth = list(raw_file.columns.values)

In [14]:
bad_indices = np.where(np.isnan(raw_file))
print(bad_indices) # should be empty

(array([], dtype=int64), array([], dtype=int64))


In [15]:
pca.n_components_

857124

In [16]:
# eignevalues: the amount of variation explained by each component
print (pca.explained_variance_ratio_)

[  1.35632023e-01   5.31992197e-03   5.26162803e-03   5.17040306e-03
   5.16188049e-03   5.13231230e-03   5.04317391e-03   3.84813009e-03
   3.82265795e-03   3.79507240e-03   3.75074981e-03   3.72242706e-03
   3.69492195e-03   3.68554416e-03   3.67620689e-03   3.65219198e-03
   3.63885320e-03   3.63357762e-03   3.62650051e-03   3.62106176e-03
   3.61093232e-03   3.60585617e-03   3.60039824e-03   3.59575169e-03
   3.59021734e-03   3.58498144e-03   3.58102976e-03   3.57250516e-03
   3.56976759e-03   3.56195257e-03   3.55872541e-03   3.55618467e-03
   3.55290739e-03   3.54592763e-03   3.53826906e-03   3.53224521e-03
   3.52925712e-03   3.52808675e-03   3.52330469e-03   3.51410959e-03
   3.51250364e-03   3.50831986e-03   3.50267095e-03   3.50070225e-03
   3.49414806e-03   3.49132055e-03   3.48928536e-03   3.48756668e-03
   3.48097211e-03   3.47872604e-03   3.47722647e-03   3.47363379e-03
   3.46899640e-03   3.46642433e-03   3.46612909e-03   3.45980317e-03
   3.45745935e-03   3.45175825e-03

In [18]:
tstart = datetime.datetime.now()

In [19]:
#this method loops through each PC and gets the list of snps having the coefficient higher than the threshold
def getPC_snps(threshold):
    consolidated_snps = {} #consolidated list of SNPs i.e list having 4 individual lists
    for i in range(0,4):#looping through each PC
        pc = pca.components_[i]
        pc_abs = abs(pc)
        num_snps = len(pc_abs)
        snp_dict = {} #dictionary key:->snp and value:->coefficient
        for j in range(0,num_snps):
            snp_dict[snps_pyth[j]] = pc_abs[j]
        top_snps,top_values = [],[]
        snp_dict_sorted = sorted(snp_dict, key=snp_dict.get, reverse=True)#sorting the dictionary
        for snp in snp_dict_sorted[:threshold]:
            coeff = snp_dict[snp]
            if snp in consolidated_snps:
                if(coeff>consolidated_snps[snp]):
                    consolidated_snps[snp] = coeff
            else:
                consolidated_snps[snp] = coeff
    return consolidated_snps

In [20]:
consolidated_snps = getPC_snps(1000) #list of filtered snps for each pc

In [21]:
sorted_consolidated_snps = sorted(consolidated_snps, key=consolidated_snps.get, reverse=True)#sorting the dictionary
top_1000_consolidated = sorted_consolidated_snps[:1000]

In [22]:
tfinish = datetime.datetime.now()
texecute = tfinish - tstart
print(texecute)

0:00:06.148000


In [23]:
fw = open("fourPCs_top1000_snps.txt","w")
fw.write("SNP\tScore\n")
for snp in top_1000_consolidated:
    fw.write(snp+"\t")
    fw.write(str(consolidated_snps[snp]))
    fw.write('\n')
fw.close()

In [24]:
threePCs_var = pca.explained_variance_ratio_[0:3]

In [25]:
sum(threePCs_var)

0.14621357327240253

In [26]:
threePCs_var

array([ 0.13563202,  0.00531992,  0.00526163])